<a href="https://colab.research.google.com/github/pinilDissanayaka/Multi-query-RAG/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
import os
from pinecone import ServerlessSpec, Pinecone
from langchain.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_pinecone import PineconeVectorStore
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
from google.colab import userdata
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
os.environ['PINECONE_API_KEY']=userdata.get('PINECORN_API_KEY')

In [4]:
llm=ChatGroq(model="mixtral-8x7b-32768",
             temperature=0.7)

In [5]:
def createIndex(indexName:str, dimension:int):
  try:
    pinecone=Pinecone()
    if indexName not in pinecone.list_indexes().names():
      pinecone.create_index(
          name=indexName,
          dimension=dimension,
          metric="cosine",
          spec=ServerlessSpec(cloud='aws',
                            region='us-east-1')
      )
      print(f"Created {indexName}")
    else:
      print(f"{indexName} already exists")
    return pinecone.describe_index(indexName)
  except Exception as e:
    print(f"Error creating index: {e}")

In [6]:
print(createIndex(indexName='multi-rag', dimension=768))

multi-rag already exists
{'deletion_protection': 'disabled',
 'dimension': 768,
 'host': 'multi-rag-4myrn7y.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'multi-rag',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}


In [7]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
webLoader=WebBaseLoader(web_path=['https://medium.com/@amanatulla1606/transformer-architecture-explained-2c49e2257b4c'])

data=webLoader.load()

print(f"Loaded {len(data)} documents")

Loaded 1 documents


In [9]:
print(data)

[Document(metadata={'source': 'https://medium.com/@amanatulla1606/transformer-architecture-explained-2c49e2257b4c', 'title': 'Transformer Architecture explained | by Amanatullah | Medium', 'description': 'Transformers are a new development in machine learning that have been making a lot of noise lately. They are incredibly good at keeping track of context, and this is why the text that they write…', 'language': 'en'}, page_content='Transformer Architecture explained | by Amanatullah | MediumOpen in appSign upSign inWriteSign upSign inTransformer Architecture explainedAmanatullah·Follow10 min read·Sep 1, 2023--11ListenShareTransformers are a new development in machine learning that have been making a lot of noise lately. They are incredibly good at keeping track of context, and this is why the text that they write makes sense. In this chapter, we will go over their architecture and how they work.Transformer models are one of the most exciting new developments in machine learning. They w

In [10]:
textSplitter=SemanticChunker(embeddings=embeddings)

splitteDocuments=textSplitter.split_documents(documents=data)

print(f"Split into {len(splitteDocuments)} chunks")

Split into 6 chunks


In [11]:
splitteDocuments[:3]

[Document(metadata={'source': 'https://medium.com/@amanatulla1606/transformer-architecture-explained-2c49e2257b4c', 'title': 'Transformer Architecture explained | by Amanatullah | Medium', 'description': 'Transformers are a new development in machine learning that have been making a lot of noise lately. They are incredibly good at keeping track of context, and this is why the text that they write…', 'language': 'en'}, page_content='Transformer Architecture explained | by Amanatullah | MediumOpen in appSign upSign inWriteSign upSign inTransformer Architecture explainedAmanatullah·Follow10 min read·Sep 1, 2023--11ListenShareTransformers are a new development in machine learning that have been making a lot of noise lately. They are incredibly good at keeping track of context, and this is why the text that they write makes sense. In this chapter, we will go over their architecture and how they work.Transformer models are one of the most exciting new developments in machine learning. They w

In [12]:
docs=[]
for doc in splitteDocuments:
  doc.page_content=doc.page_content.replace("\n", " ")
  docs.append(Document(page_content=doc.page_content))


print(f"Created {len(docs)} documents")

Created 6 documents


In [13]:
pineconeVectorStore=PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name="multi-rag"
)

In [22]:
retriever=pineconeVectorStore.as_retriever()

In [14]:
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question:

{question}"""


multiQyeryPrompt = ChatPromptTemplate.from_template(template)

print(multiQyeryPrompt)

input_variables=['question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five\ndifferent versions of the given user question to retrieve relevant documents from a vector\ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search.\nProvide these alternative questions separated by newlines. Original question:\n\n{question}'))]


In [15]:
def separateQuestions(text:str):
  return text.split("\n")

In [16]:
multiQyeryChain=multiQyeryPrompt | llm | StrOutputParser() | RunnableLambda(separateQuestions)

In [20]:
question="What is transformer architecture?"

In [21]:
multiQyeryChain.invoke(question)

['1. Can you explain the concept of a transformer architecture?',
 '2. What are the key components and workings of a transformer architecture?',
 '3. How does a transformer architecture differ from other sequential models?',
 '4. Can you provide an overview of the transformer model design and functionality?',
 '5. What makes transformer architecture significant in the field of deep learning?']

In [45]:
def getRelevantDocuments(documents:list):
  context=[]
  for document in documents:
    for subdoc in document:
      context.append(subdoc.page_content)
  context=set(context)
  return context

In [46]:
retrivalChain=multiQyeryChain | retriever.map() | RunnableLambda(getRelevantDocuments)

In [48]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""


qaPrompt = ChatPromptTemplate.from_template(template)

print(qaPrompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the following question based on this context:\n\n{context}\n\nQuestion: {question}\n'))]


In [55]:
chain= ({"context" : retrivalChain, "question" : RunnablePassthrough()}|
        qaPrompt |
        llm |
        StrOutputParser())

In [56]:
chain.invoke(question)

'Transformer architecture is a new development in machine learning that consists of a concatenation of useful components, each with its own function. It includes attention mechanisms that provide context to each word in a sentence or text by moving words closer together in the word embedding based on their relationship with other words. The transformer model is formed by many layers of transformer blocks, each containing an attention and a feedforward layer, and it predicts the next word in a sentence by outputting scores for all the words, where the highest scores are given to the words that are most likely to be next in the sentence. The softmax layer then turns these scores into probabilities, which are used to pick the next word in the text. Transformers can be used for various tasks such as writing stories, answering questions, translating languages, and even passing exams. However, they require post-training on specific datasets to perform tasks effectively.'